In [8]:
import requests
import pandas as pd
from urllib.request import Request, urlopen
from sodapy import Socrata
import json
import re

In [2]:
with open("secret.json", "r") as f:
    config = json.load(f)

token = config["TOKEN_API_OD"]
user=config["user"]
passwd=config["pw"]

In [3]:
client = Socrata('www.datos.gov.co',token,username=user,password=passwd)

In [4]:
results = client.get("mcec-87by", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [5]:
results_df

,valor,unidad,vigenciadesde,vigenciahasta
0,4012.26,COP,2023-09-09T00:00:00.000,2023-09-11T00:00:00.000
1,4045.83,COP,2023-09-08T00:00:00.000,2023-09-08T00:00:00.000
2,4093.04,COP,2023-09-07T00:00:00.000,2023-09-07T00:00:00.000
3,4089.46,COP,2023-09-06T00:00:00.000,2023-09-06T00:00:00.000
4,4063.36,COP,2023-09-02T00:00:00.000,2023-09-05T00:00:00.000
...,...,...,...,...
1995,2516.08,COP,2015-04-11T00:00:00.000,2015-04-13T00:00:00.000
1996,2494.77,COP,2015-04-10T00:00:00.000,2015-04-10T00:00:00.000
1997,2490.9,COP,2015-04-09T00:00:00.000,2015-04-09T00:00:00.000
1998,2518.05,COP,2015-04-08T00:00:00.000,2015-04-08T00:00:00.000


In [6]:
response = requests.get('https://www.datos.gov.co/resource/mcec-87by.json')
consulta=response.json()
pd.DataFrame.from_records(consulta)

,valor,unidad,vigenciadesde,vigenciahasta
0,4012.26,COP,2023-09-09T00:00:00.000,2023-09-11T00:00:00.000
1,4045.83,COP,2023-09-08T00:00:00.000,2023-09-08T00:00:00.000
2,4093.04,COP,2023-09-07T00:00:00.000,2023-09-07T00:00:00.000
3,4089.46,COP,2023-09-06T00:00:00.000,2023-09-06T00:00:00.000
4,4063.36,COP,2023-09-02T00:00:00.000,2023-09-05T00:00:00.000
...,...,...,...,...
995,3206.92,COP,2019-07-04T00:00:00.000,2019-07-05T00:00:00.000
996,3211.06,COP,2019-07-03T00:00:00.000,2019-07-03T00:00:00.000
997,3205.67,COP,2019-06-29T00:00:00.000,2019-07-02T00:00:00.000
998,3197.23,COP,2019-06-28T00:00:00.000,2019-06-28T00:00:00.000


In [ ]:
def year(x):
    patterns = [
                r"\\-.*?",  
               ]

    for pattern in patterns:
        match = re.search(pattern, x)
        if match is not None:  
            return True

    return False  

In [15]:
def extract_year(x):
    pattern = r"(\d{4})-\d{2}-\d{2}T"
    match = re.search(pattern, x)
    if match:
        return match.group(1)  # Retorna el año
    else:
        return None  # Retorna None si no hay coincidencia

In [16]:
def extract_month(x):
    pattern = r"(\d{4})-(\d{2})-\d{2}T"
    match = re.search(pattern, x)
    if match:
        return match.group(2)  # Retorna el año
    else:
        return None 

In [24]:
def extract_day(x):
    pattern = r"(\d{4})-(\d{2})-(\d{2})T"
    match = re.search(pattern, x)
    if match:
        return match.group(3)  # Retorna el año
    else:
        return None 

In [27]:
results_df['año'] = results_df['vigenciadesde'].apply(extract_year)
results_df['mes']=results_df['vigenciadesde'].apply(extract_month)
results_df['dia_inicial']=results_df['vigenciadesde'].apply(extract_day)
results_df['final']=results_df['vigenciahasta'].apply(extract_day)

In [28]:
results_df

,valor,unidad,vigenciadesde,vigenciahasta,año,mes,dia_inicial,final
0,4012.26,COP,2023-09-09T00:00:00.000,2023-09-11T00:00:00.000,2023,09,09,11
1,4045.83,COP,2023-09-08T00:00:00.000,2023-09-08T00:00:00.000,2023,09,08,08
2,4093.04,COP,2023-09-07T00:00:00.000,2023-09-07T00:00:00.000,2023,09,07,07
3,4089.46,COP,2023-09-06T00:00:00.000,2023-09-06T00:00:00.000,2023,09,06,06
4,4063.36,COP,2023-09-02T00:00:00.000,2023-09-05T00:00:00.000,2023,09,02,05
...,...,...,...,...,...,...,...,...
1995,2516.08,COP,2015-04-11T00:00:00.000,2015-04-13T00:00:00.000,2015,04,11,13
1996,2494.77,COP,2015-04-10T00:00:00.000,2015-04-10T00:00:00.000,2015,04,10,10
1997,2490.9,COP,2015-04-09T00:00:00.000,2015-04-09T00:00:00.000,2015,04,09,09
1998,2518.05,COP,2015-04-08T00:00:00.000,2015-04-08T00:00:00.000,2015,04,08,08


In [29]:
trm=results_df.copy()

In [31]:
trmFiltered=trm[['valor','mes','dia_inicial','final']]

In [32]:
trmFiltered

,valor,mes,dia_inicial,final
0,4012.26,09,09,11
1,4045.83,09,08,08
2,4093.04,09,07,07
3,4089.46,09,06,06
4,4063.36,09,02,05
...,...,...,...,...
1995,2516.08,04,11,13
1996,2494.77,04,10,10
1997,2490.9,04,09,09
1998,2518.05,04,08,08
